# 02a · LLM Decoder Alignment

Phase **2a** notebook: take a **pre-aligned vision–text encoder** (Phase 1),
freeze it, and train a **vision→LLM projector** so that a causal LLM can
generate captions from images.

This notebook:
1. Loads a Phase‑1 aligned `VisionTextAligner` checkpoint.
2. Wraps it with `MultimodalLLM` (vision → alignment → LLM decoder).
3. Uses PixMo‑Cap (or your prebuilt in‑memory loader) for image–caption pairs.
4. Trains only the **vision→LLM projector** (LLM + encoders frozen).
5. Saves Phase‑2 checkpoints which can be reused by later TRM / MoE notebooks.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# -------------------------------------------------------------
# Paths & High‑Level Config
# -------------------------------------------------------------
from pathlib import Path

# Root of your project (edit this to match your setup)
ROOT_DIR = Path.cwd()

# === Phase 1 (alignment) checkpoint ===
# Point this to the *best* Phase‑1 alignment checkpoint (from `train_alignment`).
# It should contain `vision_adapter` and `text_adapter` weights.
PHASE1_CKPT_PATH = ROOT_DIR / "checkpoints" / "phase1" / "best.pt"  # <-- EDIT

# === Phase 2 (LLM decoder alignment) output dir ===
PHASE2_RUN_NAME = "qwen2p5_prefix_alignment_v1"  # change per experiment
PHASE2_OUT_DIR = ROOT_DIR / "checkpoints" / "phase2_llm" / PHASE2_RUN_NAME
PHASE2_OUT_DIR.mkdir(parents=True, exist_ok=True)

# Optional: where your PixMo‑Cap *in‑memory* / parquet data lives (if any).
# This notebook defaults to the existing in‑memory dataloader utilities,
# but you can swap this for your parquet loader.
PIXMO_PARQUET_PATH = None  # e.g. ROOT_DIR / "data" / "pixmo_cap_train.parquet"

print("ROOT_DIR:", ROOT_DIR)
print("Phase 1 ckpt:", PHASE1_CKPT_PATH)
print("Phase 2 out dir:", PHASE2_OUT_DIR)

ROOT_DIR: /storage/ice1/1/0/vchopra37/projects/edge_glass/code_base/v2_code_base
Phase 1 ckpt: /storage/ice1/1/0/vchopra37/projects/edge_glass/code_base/v2_code_base/checkpoints/phase1/best.pt
Phase 2 out dir: /storage/ice1/1/0/vchopra37/projects/edge_glass/code_base/v2_code_base/checkpoints/phase2_llm/qwen2p5_prefix_alignment_v1


In [5]:
# -------------------------------------------------------------
# Imports & Environment Setup
# -------------------------------------------------------------
import os
import math
import random
from typing import Dict, Any

import torch
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

import numpy as np

from imports.core import AlignmentConfig, VisionTextAligner, set_seed, count_parameters
from imports.train import load_checkpoint
from imports.llm_integration import (
    LLMConfig,
    MultimodalLLM,
    create_caption_labels,
    train_multimodal_step,
)
from imports.train_with_in_memory_datasets import create_in_memory_image_dataloader

try:
    import wandb
    WANDB_AVAILABLE = True
except ImportError:
    WANDB_AVAILABLE = False

print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("W&B available:", WANDB_AVAILABLE)

Torch version: 2.9.0+cu128
CUDA available: True
W&B available: True


In [6]:
# -------------------------------------------------------------
# Experiment Hyperparameters
# -------------------------------------------------------------
class Phase2Config:
    """Lightweight config object for Phase‑2 training."""

    # Randomness
    seed = 42

    # Data
    max_samples = 20_000          # number of image–caption pairs to load
    image_size = (224, 224)
    batch_size_per_gpu = 8        # effective batch = batch_size_per_gpu * n_gpus
    num_workers = 4

    # Text / LLM
    llm_model_name = "Qwen/Qwen2.5-1.5B-Instruct"
    max_caption_tokens = 128
    num_prefix_tokens = 16        # how many soft prefix tokens from vision
    freeze_llm = True             # Phase 2a: ONLY train projector

    # Optimizer
    learning_rate = 1e-4
    weight_decay = 0.01
    max_grad_norm = 1.0
    num_epochs = 3

    # Logging
    log_every = 50
    use_wandb = True
    wandb_project = "edgeglass_phase2_llm_alignment"
    wandb_run_name = PHASE2_RUN_NAME

cfg2 = Phase2Config()

set_seed(cfg2.seed)

# Device & multi‑GPU
if torch.cuda.is_available():
    n_gpus = torch.cuda.device_count()
    device = torch.device("cuda")
else:
    n_gpus = 0
    device = torch.device("cpu")

print(f"Using device: {device}, GPUs: {n_gpus}")

Using device: cuda, GPUs: 2


In [ ]:
# -------------------------------------------------------------
# Optional: Weights & Biases Init
# -------------------------------------------------------------
wandb_run = None
if cfg2.use_wandb and WANDB_AVAILABLE:
    wandb_run = wandb.init(
        project=cfg2.wandb_project,
        name=cfg2.wandb_run_name,
        config={
            "phase": "02a_llm_decoder_alignment",
            "phase1_ckpt": str(PHASE1_CKPT_PATH),
            "llm_model": cfg2.llm_model_name,
            "batch_size_per_gpu": cfg2.batch_size_per_gpu,
            "num_prefix_tokens": cfg2.num_prefix_tokens,
            "learning_rate": cfg2.learning_rate,
            "num_epochs": cfg2.num_epochs,
        },
    )
    print("✅ WandB run initialized:", wandb_run.name)
elif cfg2.use_wandb and not WANDB_AVAILABLE:
    print("⚠️ cfg2.use_wandb=True but wandb is not installed. Logging disabled.")
else:
    print("W&B logging disabled in config.")

In [ ]:
# -------------------------------------------------------------
# Load Phase‑1 Aligned Vision–Text Model
# -------------------------------------------------------------
assert PHASE1_CKPT_PATH.is_file(), f"Phase‑1 checkpoint not found: {PHASE1_CKPT_PATH}"

print("Loading Phase‑1 checkpoint from:", PHASE1_CKPT_PATH)

# The Phase‑1 checkpoint saved the AlignmentConfig object; we reuse it.
checkpoint = torch.load(PHASE1_CKPT_PATH, map_location="cpu")
phase1_cfg: AlignmentConfig = checkpoint["config"]

# Update device in config in case you moved from CPU→GPU or vice versa
phase1_cfg.device = device

# Instantiate aligner and load adapters
aligner = VisionTextAligner(phase1_cfg).to(device)
load_checkpoint(aligner, str(PHASE1_CKPT_PATH), load_optimizer=False)

params_align = count_parameters(aligner)
print("Phase‑1 model parameters:", params_align)

# We freeze the entire aligner in Phase 2a (adapters + encoders)
for p in aligner.parameters():
    p.requires_grad = False
aligner.eval()

print("Aligner frozen for Phase‑2 training.")

In [ ]:
# -------------------------------------------------------------
# Build Phase‑2 Multimodal LLM Wrapper
# -------------------------------------------------------------
llm_cfg = LLMConfig(
    model_name=cfg2.llm_model_name,
    max_new_tokens=cfg2.max_caption_tokens,
    temperature=0.7,
    top_p=0.9,
    num_prefix_tokens=cfg2.num_prefix_tokens,
    freeze_llm=cfg2.freeze_llm,
)

print("Initializing MultimodalLLM (this will download the LLM if needed)...")
mm_model = MultimodalLLM(aligner=aligner, llm_config=llm_cfg)
mm_model.to(device)

# Multi‑GPU: wrap in DataParallel if more than 1 GPU
if torch.cuda.device_count() > 1:
    print(f"Wrapping MultimodalLLM with DataParallel across {torch.cuda.device_count()} GPUs")
    mm_model = torch.nn.DataParallel(mm_model)

# For counting params, unwrap if needed
model_for_params = mm_model.module if isinstance(mm_model, torch.nn.DataParallel) else mm_model
trainable_params = sum(p.numel() for p in model_for_params.get_trainable_params())
total_params = sum(p.numel() for p in model_for_params.parameters())

print(f"Total parameters (Phase‑2 model): {total_params:,}")
print(f"Trainable parameters (projector only): {trainable_params:,}")

In [ ]:
# -------------------------------------------------------------
# DataLoader: PixMo‑Cap In‑Memory Images + Captions
# -------------------------------------------------------------
effective_batch_size = cfg2.batch_size_per_gpu * max(1, torch.cuda.device_count())
print(f"Effective batch size: {effective_batch_size}")

# NOTE:
#   We reuse `create_in_memory_image_dataloader` which already:
#     • Loads PixMo‑Cap via HuggingFace
#     • Pre‑downloads & caches images in RAM
#     • Returns batches with keys: 'images' (PIL list), 'captions' (str list)
#   If you have a parquet‑based loader, you can swap it in here.

image_dataloader = create_in_memory_image_dataloader(
    max_samples=cfg2.max_samples,
    batch_size=effective_batch_size,
    num_workers=cfg2.num_workers,
)

len(image_dataloader)

In [ ]:
# -------------------------------------------------------------
# Phase‑2 Training Loop (Instruction Tuning with Frozen LLM)
# -------------------------------------------------------------
optimizer = torch.optim.AdamW(
    model_for_params.get_trainable_params(),
    lr=cfg2.learning_rate,
    weight_decay=cfg2.weight_decay,
)

global_step = 0
history = {"train_loss": []}

for epoch in range(cfg2.num_epochs):
    model_for_params.train()
    epoch_loss = 0.0
    n_batches = 0

    pbar = tqdm(image_dataloader, desc=f"Epoch {epoch+1}/{cfg2.num_epochs}")
    for batch_idx, batch in enumerate(pbar, start=1):
        images = batch["images"]                 # list of PIL images
        captions = batch["captions"]             # list of strings

        # Tokenize captions into input_ids, attention_mask, labels
        label_tensors = create_caption_labels(
            tokenizer=model_for_params.llm.tokenizer,
            captions=captions,
            max_length=cfg2.max_caption_tokens,
            device=device,
        )

        train_batch: Dict[str, Any] = {
            "images": images,
            "input_ids": label_tensors["input_ids"],
            "attention_mask": label_tensors["attention_mask"],
            "labels": label_tensors["labels"],
        }

        metrics = train_multimodal_step(mm_model, train_batch, optimizer)

        loss_val = metrics["loss"]
        epoch_loss += loss_val
        n_batches += 1
        global_step += 1

        if batch_idx % cfg2.log_every == 0:
            avg_loss = epoch_loss / max(1, n_batches)
            pbar.set_postfix({"loss": f"{avg_loss:.4f}"})

            if wandb_run is not None:
                wandb.log({
                    "train/loss": loss_val,
                    "train/avg_loss": avg_loss,
                    "train/epoch": epoch,
                    "global_step": global_step,
                })

    avg_epoch_loss = epoch_loss / max(1, n_batches)
    history["train_loss"].append(avg_epoch_loss)
    print(f"\n✅ Epoch {epoch+1} complete | avg loss = {avg_epoch_loss:.4f}")

    # ---------------------------------------------------------
    # Save checkpoint at end of each epoch
    # We save only the projector + configs (aligner & LLM are reusable)
    # ---------------------------------------------------------
    ckpt = {
        "epoch": epoch + 1,
        "global_step": global_step,
        "phase1_config": phase1_cfg,
        "llm_config": llm_cfg,
        "projector_state_dict": model_for_params.projector.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        "history": history,
    }

    ckpt_path = PHASE2_OUT_DIR / f"projector_epoch_{epoch+1:02d}.pt"
    torch.save(ckpt, ckpt_path)
    print("💾 Saved Phase‑2 checkpoint:", ckpt_path)

print("\n🎉 Phase‑2 LLM decoder alignment training complete!")

In [ ]:
# -------------------------------------------------------------
# Quick Qualitative Check: Generate Captions
# -------------------------------------------------------------
mm_model.eval()

sample_batch = next(iter(image_dataloader))
sample_images = sample_batch["images"][:2]
sample_captions = sample_batch["captions"][:2]

print("Reference captions:")
for i, c in enumerate(sample_captions):
    print(f"[{i}] {c[:120]}...")

print("\nGenerated captions:")
for i, img in enumerate(sample_images):
    gen = model_for_params.generate(
        images=img,
        prompt="Describe this image in one sentence:",
        max_new_tokens=cfg2.max_caption_tokens,
        temperature=0.7,
    )
    print(f"[{i}] {gen}\n")

In [ ]:
# -------------------------------------------------------------
# Save Training History & Finish
# -------------------------------------------------------------
import json

history_path = PHASE2_OUT_DIR / "train_history_phase2_llm.json"
with open(history_path, "w") as f:
    json.dump(history, f, indent=2)
print("Saved training history to:", history_path)

if wandb_run is not None:
    wandb_run.finish()
    print("Closed W&B run.")